<a href="https://colab.research.google.com/github/antmaio/TRAIL2020/blob/master/Fastai_Seance4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastai.vision.all import *

Rappel structure datablock

In [ ]:
pets = DataBlock(blocks=(ImageBlock, CategoryBlock),  # définition du TYPE d'entrée / sortie
                 get_items=get_image_files,           # définition du CHARGEMENT des entrées
                 get_y = label_func,                  # définition du CHARGEMENT des sorties
                 splitter=splitter)                   # définition de la SEPARATION des données

In [ ]:
pets = DataBlock(blocks=(ImageBlock, CategoryBlock),  
                 get_items=get_image_files,           
                 get_y = label_func,                  
                 splitter=splitter,                  
                 item_tfms=Resize(100))               # Changer la résolution les images pour obtenir une taille de 100x100 pour toutes

Rappel Dataloader

In [ ]:
# téléchargement d'une image web (chat)
# !wget https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/June_odd-eyed-cat_cropped.jpg/1216px-June_odd-eyed-cat_cropped.jpg 
# téléchargement d'une image web (chien)
# !wget https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/golden-retriever-royalty-free-image-506756303-1560962726.jpg

import glob
mes_images = glob.glob("./*.jpg")
print("les images s'appellent: ", mes_images)

from torchvision.io import read_image
im_list = []
for _,item in enumerate(mes_images):
  mon_image = read_image(item)
  im_list.append(mon_image)

print(im_list[0].shape)
print(im_list[1].shape)

# définition des catégories
cat = [1, 0] # la 1ère image est un chat et la 2ème est un chien

les images s'appellent:  ['./golden-retriever-royalty-free-image-506756303-1560962726.jpg', './1216px-June_odd-eyed-cat_cropped.jpg']
torch.Size([3, 3264, 4896])
torch.Size([3, 1024, 1216])


In [ ]:
from torchvision.transforms.functional import resize

# On définit un nouveau dataloader
def my_dataloader(liste_entree, cat_sortie):
  def pass_index(i):    # par défaut, "dblock.dataloaders" prend une liste d'indices en argument, il faut définir comment les gérer
    return i

  def get_x(i):         # comment charger les inputs
    im = resize(liste_entree[i],(512,512))
    return im

  def get_y(i):         # comment charger les outputs
    return cat_sortie[i]

  # on crée le DataBlock (note: pas besoin de splitter parce qu'on a que 2 images donc pas assez pour entrainement/validation)
  dblock = DataBlock( blocks=(TransformBlock, CategoryBlock),
                      get_items=pass_index,
                      # item_tfms=Resize(512),  #ici, on ne peut pas utiliser item_tfms car l'image est chargée par get_x
                      get_x=get_x,
                      get_y=get_y
                      )
  
  nombre_images = len(liste_entree) #la première dimension du tensor d'entrée nous donne le nombre d'éléments à traiter
  dls = dblock.dataloaders(list(range(nombre_images)), bs=2) 

  return dls

# 4) on crée le dataloader
dls = my_dataloader(im_list, cat)
print(dls.one_batch()[0].shape)

torch.Size([2, 3, 512, 512])


In [ ]:
import fastai.vision.models.tvm as tvm
dir(tvm)

In [ ]:
tvm.googlenet()

/usr/local/lib/python3.7/dist-packages/torchvision/models/googlenet.py:51: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  FutureWarning,


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [ ]:
mon_dataloader = dls
mon_model = resnet34
ma_metrique = accuracy

mon_learner = Learner(dls=mon_dataloader, model=mon_model, metrics=ma_metrique)

Les infos à mettre dans le learner sont reprises dans la doc

In [ ]:
Learner?? #Permet d'afficher la doc sur Colab

In [ ]:
mon_learner = Learner(mon_dataloader, mon_model, ma_metrique) #erreur car metrcis n'est pas un positional argument

In [ ]:
mon_learner = Learner(mon_dataloader, mon_model, metrics=ma_metrique)